by: Artyom Kurguzkin (kurg0002)

# Exploring Data
---

In [9]:
import pandas as pd

# note, the location of the data file may very, so you have to change the path to fit your work setup
df = pd.read_csv('work/GameData.csv')
df.shape

(403, 386)

In [10]:
pd.set_option('display.max_columns', 400)
df.head(5)

,GameName,String_java.lang.String,Off_int,End_int,int_int,Undefined_int,Piece_game.equipment.component.Piece,Board_game.equipment.container.board.Board,MancalaBoard_game.equipment.container.board.custom.MancalaBoard,Track_game.equipment.container.board.Track,Dice_game.equipment.container.other.Dice,Hand_game.equipment.container.other.Hand,Equipment_game.equipment.Equipment,Map_game.equipment.other.Map,Regions_game.equipment.other.Regions,All_game.functions.booleans.all.All,Can_game.functions.booleans.can.Can,Is_game.functions.booleans.is.Is,And_game.functions.booleans.math.And,Equals_game.functions.booleans.math.Equals,Ge_game.functions.booleans.math.Ge,Gt_game.functions.booleans.math.Gt,If_game.functions.booleans.math.If,Le_game.functions.booleans.math.Le,Lt_game.functions.booleans.math.Lt,Not_game.functions.booleans.math.Not,NotEqual_game.functions.booleans.math.NotEqual,Or_game.functions.booleans.math.Or,No_game.functions.booleans.no.No,Was_game.functions.booleans.was.Was,Mul_game.functions.dim.math.Mul,Difference_game.functions.directions.Difference,Directions_game.functions.directions.Directions,If_game.functions.directions.If,Div_game.functions.floats.math.Div,Mul_game.functions.floats.math.Mul,Sub_game.functions.floats.math.Sub,Square_game.functions.graph.generators.basis.square.Square,Tri_game.functions.graph.generators.basis.tri.Tri,Concentric_game.functions.graph.generators.shape.concentric.Concentric,Rectangle_game.functions.graph.generators.shape.Rectangle,Regular_game.functions.graph.generators.shape.Regular,Repeat_game.functions.graph.generators.shape.Repeat,Spiral_game.functions.graph.generators.shape.Spiral,Wedge_game.functions.graph.generators.shape.Wedge,Add_game.functions.graph.operators.Add,Hole_game.functions.graph.operators.Hole,MakeFaces_game.functions.graph.operators.MakeFaces,Merge_game.functions.graph.operators.Merge,Remove_game.functions.graph.operators.Remove,Rotate_game.functions.graph.operators.Rotate,Scale_game.functions.graph.operators.Scale,Shift_game.functions.graph.operators.Shift,SplitCrossings_game.functions.graph.operators.SplitCrossings,Array_game.functions.intArray.array.Array,Results_game.functions.intArray.math.Results,Union_game.functions.intArray.math.Union,Values_game.functions.intArray.values.Values,Ahead_game.functions.ints.board.Ahead,CentrePoint_game.functions.ints.board.CentrePoint,Column_game.functions.ints.board.Column,Coord_game.functions.ints.board.Coord,HandSite_game.functions.ints.board.HandSite,Id_game.functions.ints.board.Id,MapEntry_game.functions.ints.board.MapEntry,RegionSite_game.functions.ints.board.RegionSite,Row_game.functions.ints.board.Row,Where_game.functions.ints.board.where.Where,Count_game.functions.ints.count.Count,Face_game.functions.ints.dice.Face,Between_game.functions.ints.iterator.Between,From_game.functions.ints.iterator.From,Level_game.functions.ints.iterator.Level,Pips_game.functions.ints.iterator.Pips,Player_game.functions.ints.iterator.Player,Site_game.functions.ints.iterator.Site,To_game.functions.ints.iterator.To,Last_game.functions.ints.last.Last,Add_game.functions.ints.math.Add,Div_game.functions.ints.math.Div,If_game.functions.ints.math.If,Max_game.functions.ints.math.Max,Min_game.functions.ints.math.Min,Mod_game.functions.ints.math.Mod,Mul_game.functions.ints.math.Mul,Sub_game.functions.ints.math.Sub,Size_game.functions.ints.size.Size,TopLevel_game.functions.ints.stacking.TopLevel,Amount_game.functions.ints.state.Amount,Counter_game.functions.ints.state.Counter,Mover_game.functions.ints.state.Mover,Next_game.functions.ints.state.Next,Pot_game.functions.ints.state.Pot,Score_game.functions.ints.state.Score,State_game.functions.ints.state.State,Var_game.functions.ints.state.Var,What_game.functions.ints.state.What,Who_game.functions.ints.state.Who,TrackSite_game.functions.ints.trackSite.TrackSite,Value_game.functions.ints.value.Value,Exact_game.functions.range.math.Exact,Max_game.functions.range.math.Max,Min_game.functions.range.math.Min,Range_

last 5 collumns: labels to be predicted.
Collumns [0-381] - ludemes - game properties

## Questions

- which prediction model works best for which label?
- predict values for separate Predictions dataset

note, labels must be predicted independently (i.e., you cannot use the value from one label to help you predict the value for another label).

<br>

# Modeling
--- 